# Exercise 1

Let's build a convolutional neural network designed as follows:
- Convolutional layer with 32 3x3 filters, stride equal to 1 and 2x2 2-strided pool and ReLU activation
- Convolutional layer with 64 3x3 filters, stride equal to 1 and 2x2 2-strided pool and ReLU activation
- Fully-connected layer with 1024 neurons, with ReLU activation with dropout (keep_prob equal to 0.5)
- Fully-connected layer with 10 neurons, with softmax activation

Use GradientDescentOptimizer with learning rate equal to 0.1 for 100 iterations, and a batch size equal to 100.

In [1]:
import tensorflow as tf

def conv_weights(filters_size, channels_size, name):
    shape = filters_size + channels_size
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1), name=name)

def conv(x, W, stride, name):
    strides_shape = [1, stride, stride, 1]
    return tf.nn.conv2d(x, W, strides_shape, padding='SAME', name=name)

def pool(x, size, stride, name):
    pool_shape = [1] + size + [1]
    strides_shape = [1, stride, stride, 1]
    return tf.nn.max_pool(x, pool_shape, strides_shape, padding='SAME', name=name)

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
import math

BATCH_SIZE = 100
LEARNING_RATE = 0.0005 # BEST: GD Optimizer with lambda=0.5
ITERATIONS = 1000

x = tf.placeholder(tf.float32, [None, 28 * 28], name='input_images')
y_ = tf.placeholder(tf.float32, [None, 10], name='labels')
drop_prob = tf.placeholder(tf.float32, name='drop_prob')

input_images = tf.reshape(x, [-1, 28, 28, 1])
W1 = conv_weights([3, 3], [1, 32], 'L1_weights')
b1 = tf.Variable(tf.constant(0.1, shape=[32]), name='L1_biases')
c1 = conv(input_images, W1, stride=1, name='L1_conv')
h1 = tf.nn.relu(tf.nn.bias_add(c1, b1), name='L1_ReLU')
p1 = pool(h1, size=[2, 2], stride=2, name='L1_pool')

tf.summary.image('input', input_images)
tf.summary.histogram('L1_weights', W1)

W2 = conv_weights([3, 3], [32, 64], 'L2_weights')
b2 = tf.Variable(tf.constant(0.1, shape=[64]), name='L2_biases')
c2 = conv(p1, W2, stride=1, name='L2_conv')
h2 = tf.nn.relu(tf.nn.bias_add(c2, b2), name='L2_ReLU')
p2 = pool(h2, size=[2, 2], stride=2, name='L2_pool')

tf.summary.histogram('L2_weights', W2)

W3 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev=1 / math.sqrt(7 * 7 * 64)), name='L3_weights')
b3 = tf.Variable(tf.constant(0.1, shape=[1024]), name='L3_biases')
p2_flat = tf.reshape(p2, [-1, 7 * 7 * 64])
h3 = tf.nn.relu(tf.matmul(p2_flat, W3) + b3)
h3_drop = tf.nn.dropout(h3, 1.0 - drop_prob, name='L3_dropout')

tf.summary.histogram('L3_weights', W3)

W4 = tf.Variable(tf.truncated_normal([1024, 10], stddev=1 / math.sqrt(1024)), name='L4_weights')
b4 = tf.Variable(tf.constant(0.1, shape=[10]), name='L4_biases')
y = tf.matmul(h3_drop, W4) + b4

tf.summary.histogram('L4_weights', W4)
tf.summary.histogram('logits', y)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_), name='cross_entropy')
train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cross_entropy)
tf.summary.scalar('loss', cross_entropy)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1), name='correct_prediction')
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='train_accuracy')
tf.summary.scalar('accuracy', accuracy)

init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
merged_summaries = tf.summary.merge_all()
summary_writer = tf.summary.FileWriter('./summary/ex1', sess.graph)
sess.run(init)

In [ ]:
for i in range(ITERATIONS):
    batch_xs, batch_ys = mnist.train.next_batch(BATCH_SIZE)
    _, summary = sess.run([train_step, merged_summaries], feed_dict={x: batch_xs, y_: batch_ys, drop_prob: 0.5})
    summary_writer.add_summary(summary, i)
    
    if i % 100 == 0:
        acc = sess.run(accuracy, feed_dict={x: batch_xs, y_: batch_ys, drop_prob: 0.0})
        print('%d/%d. training accuracy = %.2f' % (i, ITERATIONS, acc))

test_acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, drop_prob: 0.0})
print('test accuracy = %.2f' % test_acc)

0/1000. training accuracy = 0.27
100/1000. training accuracy = 0.93
200/1000. training accuracy = 0.96
300/1000. training accuracy = 0.97


In [ ]:
sess.close()